# Five-Room Dungeon Generator

The Five-Room Dungeon Generator provides GMs with a way to quickly and easily generate a custom five-room dungeon that can be incorporated into their campaign.  Simply provide the notebook with a concept and the notebook will do the rest.

> [!IMPORTANT]
> You will need an [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download/dotnet/8.0) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .NET Interactive.

## Step 1 - Define Your Concept

Define your five-room dungeon concept.

In [1]:
var concept = @"
You are a game master for a Dungeons and Dragons campaign.
Please create a five-room dungeon for a party of 4 level 5 characters.
The five-room dungeon should be of medium difficulty and take place in a forest.
The party consists of a fighter, a wizard, a cleric, and a rogue.
The party is on a quest to retrieve a magical artifact from a bandit camp.
Ensure that the output includes:
* the five-room dungeon outline, 
* descriptive text for each room in the outline, 
* a list of NPCs and monsters that the party will encounter, 
* a description of the VTT map for the dungeon, and
* a rolltable of possible loot that the party can find.
Please output the generated information into a single consolidated markdown document.
";

## Step 2 - Initial Configuration


Choose whether you wish to use the OpenAI or Azure OpenAI service.

In [2]:
bool useAzureOpenAI = false;

Execute the following code which will ask a few questions and save the settings to a local
`settings.json` configuration file, under the [config](config) folder. You can
also edit the file manually if you prefer. **Please keep the file safe.**

In [3]:
#!import config/Settings.cs

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

// Uncomment this if you're using OpenAI and need to set the Org Id
await Settings.AskOrg(useAzureOpenAI);

Settings: OK: AI model configured [config/settings.json]
Settings: OK: API key configured [config/settings.json]


If you want to reset the configuration and start again, please uncomment and run the code below.
You can also edit the [config/settings.json](config/settings.json) manually if you prefer.

In [4]:
#!import config/Settings.cs

// Uncomment this line to reset your settings and delete the file from disk.
// Settings.Reset();

## Step 3 - Instantiate Services

Set up the Microsoft Semantic AI Kernel that will use for generative operations.

In [37]:

#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.11.1-preview"
#r "nuget: System.Numerics.Tensors, 8.0.0"
#r "nuget: SkiaSharp, 2.88.3"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.TextToImage;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.Numerics.Tensors;

#pragma warning disable SKEXP0001, SKEXP0010

var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
{
    builder.AddAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", azureEndpoint, apiKey);
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    builder.AddAzureOpenAITextToImage("dall-e-3", azureEndpoint, apiKey);
}
    
else
{
    builder.AddOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey, orgId);
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);
    builder.AddOpenAITextToImage(apiKey, orgId);
}

var kernel = builder.Build();

// Get AI service instance used to generate images
var dallE = kernel.GetRequiredService<ITextToImageService>();

// Get AI service instance used to extract embedding from a text
var textEmbedding = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

Installed Packages Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.11.1-preview SkiaSharp, 2.88.3 System.Numerics.Tensors, 8.0.0

Import custom plugins from plugin directory.

In [38]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "plugins");

var kernel = builder.Build();
var pluginFunctions = kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "FiveRoomDungeonPlugin"));

Set up the Handlebars Planner that will be used for orchestrating generative operations.

In [39]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

## Step 4 - Develop an Execution Plan

Have the planner generate an executiong plan based on the provided concept and kernel plugins.

In [17]:
#pragma warning disable SKEXP0060

// Generate a plan for the concept
var executionPlan  = await planner.CreatePlanAsync(kernel, concept);
Console.WriteLine(executionPlan);

{{!-- Step 0: Extract key values --}}
{{set "gameSystem" "Dungeons and Dragons"}}
{{set "partyContext" "The party consists of a fighter, a wizard, a cleric, and a rogue. They are on a quest to retrieve a magical artifact from a bandit camp."}}
{{set "location" "The dungeon is set in a forest and is of medium difficulty, suitable for a party of 4 level 5 characters."}}

{{!-- Step 1: Generate the five-room dungeon outline --}}
{{set "fiveRoomDungeonOutline" (FiveRoomDungeonPlugin-FiveRoomDungeon game_system=gameSystem input_context=(concat partyContext " " location))}}

{{!-- Step 2: Generate descriptive text for each room --}}
{{set "room1Description" (FiveRoomDungeonPlugin-RoomDescription room_number="1" input_context=fiveRoomDungeonOutline)}}
{{set "room2Description" (FiveRoomDungeonPlugin-RoomDescription room_number="2" input_context=fiveRoomDungeonOutline)}}
{{set "room3Description" (FiveRoomDungeonPlugin-RoomDescription room_number="3" input_context=fiveRoomDungeonOutline)}}
{{set

## Step 5 - Execute the Plan
Execute the plan and generate the five-room dungeon outline and room descriptions.

In [18]:
#pragma warning disable SKEXP0060

var executionPlanResult = await executionPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine(executionPlanResult.ToString());

# Five Room Dungeon

## Dungeon Outline
## Room 1: Entrance and Guardian
* **Objective:** Create an initial challenge.
* **Design:** The entrance to the bandit camp is hidden within a dense part of the forest. The path is guarded by a pair of bandit scouts and a trained wolf. The scouts are alert and have set up a tripwire trap that will alert the camp if triggered.

**Description:**
The party approaches a dense thicket, where the path narrows and becomes overgrown. Ahead, they can see two bandit scouts patrolling with a wolf. The scouts are wary and have set up a tripwire connected to a bell hidden in the bushes.

**Challenge:**
- The rogue can attempt to disarm the tripwire trap (DC 15 Dexterity check with Thieves&#39; Tools).
- The party can choose to engage the scouts and the wolf in combat or try to sneak past them (DC 14 Stealth check for the whole party).

## Room 2: Puzzle or Roleplaying Challenge
* **Objective:** Engage players with non-combat interaction.
* **Design:** After 

Save the output to the filesystem as `five-room-dungeon.md` in the `output` directory.

In [24]:
// Delete all contents of the directory
var outputDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "output");

// Replace "&#39;"" with "'"
var markdownContent = executionPlanResult.Replace("&#39;", "'");

// Write the variable executionPlanResults to a file
File.WriteAllText(Path.Combine(outputDirectory, "five-room-dungeon.md"), markdownContent);

## Step 6 - Codify Data

The generated content will be more useful to us if we can codify it into a data format that can be consumed by other systems.

In [40]:
// Construct arguments
var arguments = new KernelArguments() { ["input_context"] = markdownContent };

// Run the Function called Joke
var jsonContent5RD = await kernel.InvokeAsync(pluginFunctions["JSONFormatFiveRoomDungeon"], arguments);
var jsonContentLoot = await kernel.InvokeAsync(pluginFunctions["JSONFormatLootTable"], arguments);
var jsonContentNPCs = await kernel.InvokeAsync(pluginFunctions["JSONFormatNPCs"], arguments);

// Save the codified content to file.
File.WriteAllText(Path.Combine(outputDirectory, "five-room-dungeon.json"), jsonContent5RD.ToString());
File.WriteAllText(Path.Combine(outputDirectory, "loot-table.json"), jsonContentLoot.ToString());
File.WriteAllText(Path.Combine(outputDirectory, "npcs.json"), jsonContentNPCs.ToString());

## Step 99 - Generate a Map (Experimental)

Generate a map for the five-room dungeon.

In [ ]:
#pragma warning disable SKEXP0001

var prompt = @"
You are a map maker's assistant. You need to describe a map to a map maker who will then make it.
Given the five-room dungeon outline provided below, describe the map that the map maker should create.
Describe only the layout of the map.
Do not include any details about the NPCs or monsters that the party will encounter.
Do not include any details about the five-room dungeon plot.

```
{{$input_context}}
```
";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 1024,
    Temperature = 1
};

// Create a semantic function that generate a random image description.
var genImgDescription = kernel.CreateFunctionFromPrompt(prompt, executionSettings);

var random = new Random().Next(0, 200);
var imageDescriptionResult = await kernel.InvokeAsync(genImgDescription, new() { ["input_context"] = executionPlanResult.ToString() });
var imageDescription = "A TTRPG battle map for the following five-room dungeon:\n\n  " + imageDescriptionResult.ToString();
imageDescription = imageDescription.Substring(0, imageDescription.Length > 1000 ? 1000 : imageDescription.Length);
Console.WriteLine("Generated Map Description:");
Console.WriteLine(imageDescription);

// Use DALL-E 3 to generate an image. OpenAI in this case returns a URL (though you can ask to return a base64 image)
var imageUrl = await dallE.GenerateImageAsync(imageDescription.Trim(), 1024, 1024);

await SkiaUtils.ShowImage(imageUrl, 1024, 1024);